In [ ]:
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")


import os
import re
from pathlib import Path
from tempfile import TemporaryDirectory
from time import sleep

import matplotlib.pyplot as plt
import MDAnalysis as mda
import numpy as np
import pandas as pd
from MDAnalysis.analysis import msd, rms
from mycolorpy import colorlist as mcp
from scipy.stats import linregress

from base import RD_MOLECULES, Mols

plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('font', size=10)
plt.rc('axes', titlesize=16)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=10)
plt.rcParams["figure.figsize"] = (10, 7)

# Traj analyse

In [ ]:
def get_solve_dir(work_dir: str):
    paths = []
    result_dir = Path(work_dir)
    for system_sir in os.listdir(result_dir):
        for T_dir in os.listdir(result_dir / system_sir):
            paths.append(result_dir / system_sir / T_dir)
    return sorted(paths)


def regress(data: pd.DataFrame):
    time, MSD = data.iloc[:, 0], data.iloc[:, 1]
    regression = linregress(time, MSD)
    S0 = regression.intercept
    S = regression.slope
    return S, S0


def strict(df: pd.DataFrame):
    slopes = []
    for i in np.arange(0, 200, 20):
        slopes.append((df.iloc[i, 0], regress(df.loc[i : i + 20])[0], i))
    slopes = np.array(slopes)
    need_slopes = slopes[(slopes[:, 1] - slopes[:, 1].mean()) / (slopes[:, 1].mean() + 1e-9) < 0.2]
    i0, i1 = need_slopes[0][2], need_slopes[-1][2]
    return df.loc[i0:i1], (i0, i1)

In [ ]:
class D_results:
    D: float = None
    variation: float = 0
    MSD: pd.DataFrame = None
    unit: str = 'nm2/s'
    S: float = 0
    S0: float = 0
    start: float = 0
    end: float = 0

    def __init__(self, compound: Mols):
        self.compound = compound

    def __repr__(self):
        return f'<D_results: {self.D: .3e} (+/- {self.variation: .1e}) {self.unit}>'

    def MSD_mda(self, pdb: Path, traj: Path):
        u = mda.Universe(pdb, traj)
        selection = f'resname {self.compound.value}'

        m = msd.EinsteinMSD(u, select=selection)
        m.run()

        time = m.times
        MSD = np.mean(m.results['msds_by_particle'], axis=1)
        df, (i0, i1) = strict(pd.DataFrame({'Time': time, self.compound.name: MSD}))  # ps,  # nm
        S, S0 = regress(df)
        D = S / 6 * 1e12 / 1e14  # cm2/s
        self.D = D
        self.variation = 0
        self.MSD = df
        self.S = S
        self.S0 = S0
        self.start = i0
        self.end = i1

    def get_MSD_gmx(self, system: Path, traj: Path):
        with TemporaryDirectory() as temp_dir:
            temp_dir = Path(temp_dir)
            index_path = temp_dir / f'{self.compound.value}.ndx'
            xvg_path = temp_dir / f'{self.compound.value}.xvg'
            mol_path = temp_dir / f'{self.compound.value}_mol.xvg'

            command = f"""
                gmx -quiet select \
                -s {system} \
                -select {self.compound.value} \
                -on {index_path}

                gmx -quiet msd \
                -s {system} \
                -f {traj} \
                -n {index_path} \
                -o {xvg_path} \
                -beginfit 50 \
                -endfit 300 \
                -rmcomm
                """
            os.system(command)
            sleep(1)
            self._parse_xvg(xvg_path)
        self.S, self.S0 = regress(self.MSD[int(self.start / 2) : int(self.end / 2)])

    @staticmethod
    def _RMSD_mda(u, compound: Mols):
        residue = compound.value
        selection = f'resname {residue}'
        m = rms.RMSD(u, u, selection)
        m.run()
        return pd.DataFrame(m.results.rmsd[:, 1:], columns=['Time', residue])

    def _parse_xvg(self, xvg_path):
        with open(xvg_path) as file:
            data = file.readlines()
        D_pattern = r'D\[\s*(?P<name>\w+)\s*\] = (?P<value>[\d\.]+)\s*\(\+\/\- (?P<var>.+)\s*\)\s*\(\s*(?P<unit>.+)\s*\)'
        time_pattern = r'fitted from time\s(?P<start>\d+)\s*to\s*(?P<end>\d+).*'
        MSD_pattern = r'\s*(?P<time>\d+)\s*(?P<MSD>[\d\.]+).*'

        MSD = []
        D_flag = False
        for line in data:
            D_line = re.search(string=line, pattern=D_pattern)
            MSD_line = re.search(string=line, pattern=MSD_pattern)
            fit = re.search(string=line, pattern=time_pattern)

            if D_line:
                value = D_line.group('unit')
                scale, self.unit = [part.strip() for part in value.split(' ')]
                scale = float(scale)
                self.D = float(D_line.group('value')) * scale
                self.variation = float(D_line.group('var'))
                D_flag = True

            elif MSD_line and D_flag:
                time, msd = MSD_line.groups()
                MSD.append((int(time), float(msd)))

            elif fit:
                self.start = float(fit.group('start'))
                self.end = float(fit.group('end'))

        self.MSD = pd.DataFrame(np.array(MSD), columns=['Time', self.compound.value])

## Test


In [ ]:
# pdb = Path(r'gromacs/workspace/solvs/butanol_100_700/30/box.gro')
system = Path('gromacs/workspace/solvs/ocm_38_1539/30/5MD/system.tpr')
traj = Path(r'gromacs/workspace/solvs/ocm_38_1539/30/5MD/md.xtc')

In [ ]:
but=D_results(Mols.butanol)
but.get_MSD_gmx(system=system, traj=traj)

In [ ]:
ocm=D_results(Mols.ocm)
ocm.get_MSD_gmx(system=system, traj=traj)

In [ ]:
fig, ax = plt.subplots()
ax.set_ylabel('MSD [nm2]')
ax.set_xlabel('Time [ps]')

ax.scatter(
    a.MSD.iloc[:, 0],
    a.MSD.iloc[:, 1],
    label=f'{compound.name:7} [{a.D: 9.2e} cm2/s]',
    c='g',
    s=2,
)
ax.plot(a.MSD.iloc[:, 0], (a.S * a.MSD.iloc[:, 0] + a.S0), 'g')

# time_but = df_but['Time']

# ax.scatter(
#     time_but ,
#     df_but.iloc[:, 1] ,
#     label=f'butanol [{D_but: 9.2f} cm2/day]',
#     c='b',
#     s=2,
# )
# ax.plot(time_but , (S_but * time_but + S0_but) , 'b-')
ax.legend()

# ax.set_title(f"{compound} - {x} butanol :{T} C")
# fig.savefig(f"MSD_old.jpg", dpi=600)

In [ ]:
# for directory in get_solve_dir(Path('gromacs/workspace/solvs')):
#     name = r'.traj.xtc_offsets.lock'
#     name2 = r'.traj.xtc_offsets.npz'
#     os.remove(directory / name)
#     os.remove(directory / name2)

In [ ]:
solves_path=Path('gromacs/workspace/solvs')
plots_path=Path('gromacs/workspace/plots')
for directory in get_solve_dir(solves_path):
    compound, x, rho = directory.parent.stem.split('_')
    T = directory.stem
    compound = Mols[compound]

    system = directory / '5MD/system.tpr'
    traj = directory / '5MD/md.xtc'

    comp = D_results(compound)
    comp.get_MSD_gmx(system=system, traj=traj)
    but = D_results(Mols.butanol)
    if x != '0':
        but.get_MSD_gmx(system=system, traj=traj)

## loop

In [ ]:
solves_path=Path('gromacs/workspace/solvs')
plots_path=Path('gromacs/workspace/plots')

In [ ]:
datas = []
for directory in get_solve_dir(solves_path):
    compound, x, rho = directory.parent.stem.split('_')
    T = directory.stem
    compound = Mols[compound]

    system = directory / '5MD/system.tpr'
    traj = directory / '5MD/md.xtc'

    comp = D_results(compound)
    comp.get_MSD_gmx(system=system, traj=traj)
    but = D_results(Mols.butanol)
    if x != '0':
        but.get_MSD_gmx(system=system, traj=traj)

    #
    # fig, ax = plt.subplots()
    # ax.set_title(f"{compound.name} - {x} butanol :{T} C")
    # ax.set_ylabel('MSD [nm2]')
    # ax.set_xlabel('Time [ps]')

    # ax.scatter(
    #     comp.MSD.iloc[:, 0],
    #     comp.MSD.iloc[:, 1],
    #     label=f'{compound.name:7} [{comp.D: 9.2e} cm2/s]',
    #     c='g',
    #     s=2,
    # )
    # ax.plot(comp.MSD.iloc[:, 0], (comp.S * comp.MSD.iloc[:, 0] + comp.S0), 'g')

    # if x != '0':
    #     ax.scatter(
    #         but.MSD.iloc[:, 0],
    #         but.MSD.iloc[:, 1],
    #         label=f'{Mols.butanol.name:7} [{but.D: 9.2e} cm2/s]',
    #         c='b',
    #         s=2,
    #     )
    #     ax.plot(but.MSD.iloc[:, 0], (but.S * but.MSD.iloc[:, 0] + but.S0), 'b')
    # ax.legend()
    # plots_path.mkdir(parents=True, exist_ok=True)
    # fig.savefig(plots_path / f"{compound.name}_{x}_{T}.jpg", dpi=600)

    datas.append(
        {
            'compound': compound.name,
            'x': int(x),
            'Temperature': int(T),
            'D_comp': comp.D,
            'D_but': but.D,
        }
    )
D_df = pd.DataFrame(datas)
D_df.to_csv(Path(plots_path).parent / 'res.csv')

# D analyse

In [ ]:
D_df = pd.read_csv(r'gromacs/workspace/res.csv',index_col=0)

In [ ]:
D_df[D_df['compound']=='ocm']

In [ ]:
# D
colors_list = mcp.gen_color(
    cmap="turbo",
    n=len(D_df['compound'].unique()),
)
i = 0
for compound in Mols:
    df_sub = D_df[D_df['compound'] == compound.name]
    if len(df_sub) == 0:
        continue

    # fig, ax = plt.subplots(num="DT")

    for [x], df in df_sub.groupby(by=['x']):
        fig, ax = plt.subplots(num="DT")
        data_x = df.sort_values(by=['Temperature'], ascending=False)
        ax.plot(
            data_x['Temperature'],
            data_x['D_comp'],
            label=f"{compound.name}: {100-x} %",
            c=colors_list[i],
        )
        ax.plot(
            data_x['Temperature'],
            data_x['D_but'],
            linestyle='dashed',
            label=f"Butanol: {x} %",
            c=colors_list[i],
        )

        ax.set_xlabel("Temperature [C]")
        ax.set_ylabel("D cm2/s")
        ax.legend()
        plt.show()
    i += 1

# Combine results

In [ ]:
D = pd.read_csv(r'gromacs/workspace/res.csv', index_col=0)
V = pd.read_csv(r'vixcosity/results.csv', index_col=0)[
    ['x', 'Temperature', 'rho', 'w_mass', 'w_vol', 'Viscosity', 'compound']
]
V['x'] = V['x'].astype(int)

In [ ]:
def calculate_h_comp(r: pd.Series):
    T = r['Temperature']+273.15
    v = r['Viscosity']
    D_comp = r['D_comp']
    return T / (v * D_comp)


def calculate_h_but(r: pd.Series):
    T = r['Temperature']+273.15
    v = r['Viscosity']
    D_but = r['D_but']
    return T / (v * D_but) if D_but else np.nan

M=pd.merge(D,V,on=['x','Temperature','compound'],how='inner')
M = M[M['Temperature']!=20]
M['H_comp']= M.apply(calculate_h_comp,axis=1)
M['H_but']= M.apply(calculate_h_but,axis=1)

In [ ]:
colors_list = mcp.gen_color(
    cmap="turbo",
    n=len(M['compound'].unique()),
)
i = 0
d_dict = {}
for name, group in M.groupby(by='compound'):
    H_comp = np.array(group['H_comp'])
    # if name == 'butanol':
    #     H_but = np.array(M['D_but'])
    #     H_comp = np.array([*H_comp, *H_but])[:]

    H0 = np.median(H_comp)
    d_dict[name] = H0
    fig, ax = plt.subplots(num="H")
    ax.plot(
        H_comp,
        c=colors_list[i],
        label=f"Component: {name} [{H0: .2f}]",
    )
    ax.plot(
        [H0] * len(H_comp),
        c=colors_list[i],
        linestyle='dashed',
    )
    ax.legend()
    plt.show()
    i += 1

In [ ]:
def calculate_D(r):
    v = r['Viscosity']
    T = r['Temperature']+273
    comp = r['compound']
    h = d_dict[comp]
    return T / v / h


def calculate_D_but(r):
    v = r['Viscosity']
    T = r['Temperature']+273
    h = d_dict['butanol']
    return T / v / h

M['D_comp0']=M['D_comp']
M['D_comp'] = M.apply(calculate_D, axis=1)
M['D_but0']=M['D_but']
M['D_but'] = M.apply(calculate_D_but, axis=1)
M.head()

In [ ]:
M.iloc[-5:]

In [ ]:
plt.plot(
    np.log(M['D_comp']/M['D_comp0'])
)
# plt.plot(
#     M['D_comp0']
# )